In [1]:
pip install librosa==0.8.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 203.8/203.8 kB 4.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 47.3 MB/s eta 0:00:00
  Attempting uninstall: librosa
    Found existing installation: librosa 0.10.0.post2
    Uninstalling librosa-0.10.0.post2:
      Successfully uninstalled librosa-0.10.0.post2


In [11]:
!pip install pretty_midi


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 29.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.1/51.1 kB 5.2 MB/s eta 0:00:00
  Created wheel for pretty_midi: filename=pretty_midi-0.2.10-py3-none-any.whl size=5592303 sha256=7bce1a495e30d7f12c093166f8b9097a9a5d8dea33644b1283b4877fce72f235
  Stored in directory: /root/.cache/pip/wheels/cd/a5/30/7b8b7f58709f5150f67f98fde4b891ebf0be9ef07a8af49f25
Successfully built pretty_midi


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
m1 = "/content/drive/My Drive/Colab Notebooks/Music/1.mp3"
m2 = "/content/drive/My Drive/Colab Notebooks/Music/3.mp3"

In [4]:
import librosa

# Load the music dataset (audio or MIDI)
# For audio dataset
audio_path1 = m1
audio_path2 = m2

# Load the first music file
y1, sr1 = librosa.load(audio_path1)

# Load the second music file
y2, sr2 = librosa.load(audio_path2)

# Extract beats from the first music file
# For audio dataset
tempo1, beats1 = librosa.beat.beat_track(y1, sr=sr1)

# Extract beats from the second music file
# For audio dataset
tempo2, beats2 = librosa.beat.beat_track(y2, sr=sr2)

# Convert beats into a suitable format for the RNN model
# Option 1: Sequence of time intervals
time_intervals1 = librosa.frames_to_time(beats1, sr=sr1)
time_intervals2 = librosa.frames_to_time(beats2, sr=sr2)

# Option 2: Symbolic representation
symbolic_beats1 = [1 if t in beats1 else 0 for t in range(len(y1))]
symbolic_beats2 = [1 if t in beats2 else 0 for t in range(len(y2))]

# Print the results
print("Tempo 1:", tempo1)
print("Beats 1:", beats1)
print("Time intervals 1:", time_intervals1)
# print("Symbolic representation 1:", symbolic_beats1)

print("Tempo 2:", tempo2)
print("Beats 2:", beats2)
print("Time intervals 2:", time_intervals2)
# print("Symbolic representation 2:", symbolic_beats2)


Tempo 1: 117.45383522727273
Beats 1: [   5   26   47   70   94  119  141  162  185  208  232  255  277  299
  321  344  370  392  414  436  457  479  501  522  543  564  586  608
  629  650  672  694  715  737  759  783  808  830  852  875  899  923
  945  966  987 1009 1032 1053 1074 1095 1117 1139 1161 1185 1210 1232
 1253 1275 1296 1318 1340 1362 1383 1404 1425 1446 1469 1491 1512 1533
 1555 1576 1598 1620 1641 1669 1691 1713 1735 1756 1778 1800 1821 1843
 1864 1885 1907 1932 1957 1978 2000 2022 2047 2072 2093 2115 2137 2162
 2187 2208 2229 2252 2276 2301 2323 2344 2366 2391 2416 2437 2459 2481
 2506 2531 2552 2574 2596 2620 2645 2667 2689 2711 2733 2754 2776 2797
 2818 2839 2861 2883 2904 2926 2948 2969 2990 3012 3033 3055 3077 3105
 3127 3148 3170 3191 3213 3235 3256 3278 3299 3321 3343 3364 3386 3408
 3429 3450 3471 3493 3515 3540 3565 3586 3608 3630 3651 3673 3695 3716
 3737 3758 3780 3802 3823 3845 3867 3888 3909 3931 3952 3975 3998 4022
 4045 4067 4089 4110 4133 4154 4175 4196

In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
import numpy as np

# Preprocess the input beats for training
input_beats1 = np.array(symbolic_beats1)
input_beats1 = np.expand_dims(input_beats1, axis=1)

input_beats2 = np.array(symbolic_beats2)
input_beats2 = np.expand_dims(input_beats2, axis=1)

# Define the RNN model architecture
model = Sequential()
model.add(LSTM(64, input_shape=(len(input_beats1[0]), 1)))
model.add(Dense(len(input_beats1[0]), activation='sigmoid'))

# Split the dataset into training and evaluation sets
X_train1, X_eval1, y_train1, y_eval1 = train_test_split(input_beats1, input_beats1, test_size=0.2, random_state=42)
X_train2, X_eval2, y_train2, y_eval2 = train_test_split(input_beats2, input_beats2, test_size=0.2, random_state=42)

# Compile and train the RNN model on first beat sequence
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X_train1, y_train1, epochs=10, batch_size=32, validation_data=(X_eval1, y_eval1))

# Evaluate the performance of the model on first beat sequence
loss1 = model.evaluate(X_eval1, y_eval1)
print("Evaluation Loss 1:", loss1)

# Compile and train the RNN model on second beat sequence
model.compile(loss='binary_crossentropy', optimizer='adam')
model.fit(X_train2, y_train2, epochs=10, batch_size=32, validation_data=(X_eval2, y_eval2))

# Evaluate the performance of the model on second beat sequence
loss2 = model.evaluate(X_eval2, y_eval2)
print("Evaluation Loss 2:", loss2)

Epoch 1/10
125696/125696 [==============================] - 445s 4ms/step - loss: 8.4151e-04 - val_loss: 1.0045e-10
Epoch 2/10
125696/125696 [==============================] - 466s 4ms/step - loss: 6.6993e-11 - val_loss: 4.3511e-11
Epoch 3/10
125696/125696 [==============================] - 422s 3ms/step - loss: 3.5609e-11 - val_loss: 2.8850e-11
Epoch 4/10
125696/125696 [==============================] - 429s 3ms/step - loss: 2.6137e-11 - val_loss: 2.2316e-11
Epoch 5/10
125696/125696 [==============================] - 469s 4ms/step - loss: 2.0625e-11 - val_loss: 1.8326e-11
Epoch 6/10
125696/125696 [==============================] - 426s 3ms/step - loss: 1.7585e-11 - val_loss: 1.5916e-11
Epoch 7/10
125696/125696 [==============================] - 465s 4ms/step - loss: 1.5433e-11 - val_loss: 1.4020e-11
Epoch 8/10
125696/125696 [==============================] - 464s 4ms/step - loss: 1.3618e-11 - val_loss: 1.2397e-11
Epoch 9/10
125696/125696 [==============================] - 427s 3ms/ste

In [12]:
# Prepare seed beats as input for the model
seed_beats = np.random.randint(0, 2, size=(1, len(input_beats1[0]), 1))

# Generate new beats using the trained RNN model
generated_beats = []
num_steps = 100  # Number of steps to generate
for _ in range(num_steps):
    # Reshape the seed beats for input to the model
    seed_beats_reshaped = np.reshape(seed_beats, (1, len(seed_beats), 1))
    # Predict the next beat using the model
    next_beat = model.predict(seed_beats_reshaped)
    # Append the predicted beat to the generated sequence
    generated_beats.append(next_beat)
    # Update the seed by removing the first beat and adding the predicted beat
    seed_beats = seed_beats[1:]
    seed_beats = np.append(seed_beats, next_beat)

# Convert the generated beats back into a human-readable format
# You can choose the format based on your requirements, such as MIDI or audio file

# MIDI example:
import pretty_midi

# Create a PrettyMIDI object
midi = pretty_midi.PrettyMIDI()
# Create an Instrument object
instrument = pretty_midi.Instrument(program=0)
# Set the beats as notes in the Instrument object
for beat in generated_beats:
    note = pretty_midi.Note(
        velocity=100,
        pitch=60,  # Example pitch value, you can modify based on your needs
        start=0.0,  # Example start time, you can modify based on your needs
        end=0.1  # Example end time, you can modify based on your needs
    )
    instrument.notes.append(note)
# Add the Instrument object to the PrettyMIDI object
midi.instruments.append(instrument)
# Save the MIDI file
midi.write('generated_music.mid')

# Audio file example:
# You can use a library like librosa to convert the beats into an audio waveform
# and save it as an audio file based on your needs


1/1 [==============================] - 0s 30ms/step


In [1]:
# Prepare seed beats as input for the model
seed_beats = X_eval[0]  # You can use a randomly generated seed or select from the evaluation set

# Generate new beats using the trained RNN model
generated_beats = []
num_steps = 100  # Number of steps to generate
for _ in range(num_steps):
    # Reshape the seed beats for input to the model
    seed_beats_reshaped = np.reshape(seed_beats, (1, len(seed_beats), 1))
    # Predict the next beat using the model
    next_beat = model.predict(seed_beats_reshaped)
    # Append the predicted beat to the generated sequence
    generated_beats.append(next_beat)
    # Update the seed by removing the first beat and adding the predicted beat
    seed_beats = seed_beats[1:]
    seed_beats = np.append(seed_beats, next_beat)

# Convert the generated beats back into a human-readable format
# You can choose the format based on your requirements, such as MIDI or audio file

# MIDI example:
import pretty_midi

# Create a PrettyMIDI object
midi = pretty_midi.PrettyMIDI()
# Create an Instrument object
instrument = pretty_midi.Instrument(program=0)
# Set the beats as notes in the Instrument object
for beat in generated_beats:
    note = pretty_midi.Note(
        velocity=100,
        pitch=60,  # Example pitch value, you can modify based on your needs
        start=0.0,  # Example start time, you can modify based on your needs
        end=0.1  # Example end time, you can modify based on your needs
    )
    instrument.notes.append(note)
# Add the Instrument object to the PrettyMIDI object
midi.instruments.append(instrument)
# Save the MIDI file
midi.write('generated_music.mid')

# Audio file example:
# You can use a library like librosa to convert the beats into an audio waveform
# and save it as an audio file based on your needs

NameError: ignored